In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Combine DataFrames

In [66]:
home_page = pd.read_csv('home_page_table.csv')
user = pd.read_csv('user_table.csv')
payment = pd.read_csv('payment_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
search = pd.read_csv('search_page_table.csv')

In [68]:
user['date'] = pd.to_datetime(user['date'])

user['day_of_week'] = user['date'].dt.day_name()
user['day_of_month'] = user['date'].dt.day
user['month'] = user['date'].dt.month

In [70]:
user.head(2)

,user_id,date,device,sex,day_of_week,day_of_month,month
0,450007,2015-02-28,Desktop,Female,Saturday,28,2
1,756838,2015-01-13,Desktop,Male,Tuesday,13,1


In [72]:
df = user.merge(home_page, on='user_id', how='left').merge(search, on='user_id', how='left').merge(payment, on='user_id', how='left').merge(payment_confirmation, on='user_id', how='left')
df.columns = ['user_id', 'date', 
              'device', 'sex', 
              'day_of_week','day_of_month',
              'month','home_page',
              'search_page','payment_page',
              'payment_confirmation_page']

In [73]:
d = {'home_page':1, 'search_page':1,'payment_page':1,'payment_confirmation_page':1}
df.replace(d, inplace=True)
df.fillna(value=0,inplace=True)

In [74]:
df[['home_page','search_page','payment_page','payment_confirmation_page']] = df[['home_page','search_page','payment_page','payment_confirmation_page']].astype(int)

In [75]:
df.head(10)

,user_id,date,device,sex,day_of_week,day_of_month,month,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,Saturday,28,2,1,0,0,0
1,756838,2015-01-13,Desktop,Male,Tuesday,13,1,1,0,0,0
2,568983,2015-04-09,Desktop,Male,Thursday,9,4,1,1,0,0
3,190794,2015-02-18,Desktop,Female,Wednesday,18,2,1,1,0,0
4,537909,2015-01-15,Desktop,Male,Thursday,15,1,1,0,0,0
5,993454,2015-03-03,Desktop,Male,Tuesday,3,3,1,0,0,0
6,377150,2015-04-15,Desktop,Female,Wednesday,15,4,1,1,0,0
7,137258,2015-01-27,Mobile,Male,Tuesday,27,1,1,0,0,0
8,608249,2015-04-12,Desktop,Female,Sunday,12,4,1,0,0,0
9,615615,2015-02-24,Mobile,Female,Tuesday,24,2,1,1,0,0


### Visual Insights

### One Hot Encode